In [16]:
import pandas as pd
import numpy as np

In [21]:
sol=pd.read_csv('SMILES-DATA.csv')
sol

,SMILES,RESULT
0,C=CC(=O)NC(=O)NC1(CC2CC3CC(C2)C1)C3,0
1,CC=1C=CC(=CC1)C(=O)C=2OC=CC2NC(=O)C=C,0
2,CN(C)C1=NC(Cl)=C(CNC(=O)C=C)S1,0
3,C=CC(=O)N1CN(CC=2C=CC=CC2)C(=O)C1,0
4,C=CC(=O)NCC(=O)N1CCC=2C=CC=CC2C1,0
...,...,...
1823,CS(=O)(=O)NCC1COCCN1C(=O)C=C,1
1824,COC=1C=CC=C(C1)C2CN(CCO2)C(=O)C=C,1
1825,C[C@@H](NC(=O)C=C)C=1C=NC=CN1,1
1826,CC(C)N1C=C(N=N1)C2=CCCN(CC2)C(=O)C=C,1


In [22]:
sol.SMILES

0         C=CC(=O)NC(=O)NC1(CC2CC3CC(C2)C1)C3
1       CC=1C=CC(=CC1)C(=O)C=2OC=CC2NC(=O)C=C
2              CN(C)C1=NC(Cl)=C(CNC(=O)C=C)S1
3           C=CC(=O)N1CN(CC=2C=CC=CC2)C(=O)C1
4            C=CC(=O)NCC(=O)N1CCC=2C=CC=CC2C1
                        ...                  
1823             CS(=O)(=O)NCC1COCCN1C(=O)C=C
1824        COC=1C=CC=C(C1)C2CN(CCO2)C(=O)C=C
1825            C[C@@H](NC(=O)C=C)C=1C=NC=CN1
1826     CC(C)N1C=C(N=N1)C2=CCCN(CC2)C(=O)C=C
1827            C=CC(=O)N1CCC=2C1=CC=CC2C3CC3
Name: SMILES, Length: 1828, dtype: object

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.metrics import precision_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split



######################################    Initialize wandb  ######################################################
######################################    Initialize wandb  ######################################################
######################################    Initialize wandb  ######################################################



# Load a ChemBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k')


max_length = 128  # Define the maximum sequence length

print(tokenizer)

# Preprocess the data for BERT (tokenization, padding, etc.)
tokens = tokenizer(sol['SMILES'].tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

# Extract input_ids and attention_mask from tokens
input_ids = tokens['input_ids']
attention_masks = tokens['attention_mask']

# Convert 'RESULT' column to a tensor
labels = torch.tensor(sol['RESULT'].values)

print('INPUT', input_ids)
print('attention_masks', attention_masks)
print('labels', labels)

RobertaTokenizerFast(name_or_path='seyonec/PubChem10M_SMILES_BPE_450k', vocab_size=7924, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	4: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}
INPUT tensor([[  0,  39,  33,  ...,   1,   1,   1],
       

In [24]:

# Split the data into train and test sets

train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, labels, test_size=0.2, random_state=42)
train_masks, test_masks, _, _ = train_test_split(attention_masks, labels, test_size=0.2, random_state=42)


In [25]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDatasets
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)

# Create DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)


In [ ]:


# Load a ChemBERTa model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k', num_labels=2)


# Define optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=5e-5)

# Fine-tune BERT for your task
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)  # Move batch to GPU
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Average Loss: {total_loss/len(train_dataloader)}")

# Evaluation loop
model.eval()
predictions, true_labels = [], []
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = batch[2].to('cpu').numpy()
    predictions.extend(np.argmax(logits, axis=1).flatten())
    true_labels.extend(label_ids.flatten())

# Calculate precision
precision = precision_score(true_labels, predictions, average='binary')
print(f"Precision: {precision}")

# Save the model
torch.save(model.state_dict(), 'chemberta_model.pth')



Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\babak\AppData\Roaming\Python\Python310\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Average Loss: 0.3972979843940424
Epoch 2/10, Average Loss: 0.3590104633780277
Epoch 3/10, Average Loss: 0.3445495361786174


In [17]:
sol2=pd.read_csv('SMILES-DATA - Equal-1 - Validation.csv')
sol2


,SMILES,RESULT
0,C=CC(=O)NC(=O)NC1(CC2CC3CC(C2)C1)C3,0
1,CC=1C=CC(=CC1)C(=O)C=2OC=CC2NC(=O)C=C,0
2,CN(C)C1=NC(Cl)=C(CNC(=O)C=C)S1,0
3,C=CC(=O)N1CN(CC=2C=CC=CC2)C(=O)C1,0
4,C=CC(=O)NCC(=O)N1CCC=2C=CC=CC2C1,0
5,CN(C)S(=O)(=O)C=1C=CC=2CN(CC2C1)C(=O)C=C,0
6,CN1N=CC=2C=CC(CNC(=O)C=C)=CC12,0
7,CC(NC(=O)C=C)C(=O)N1CCC=2C=CC=CC12,0
8,C=CC(=O)N1CCN(CC1)S(=O)(=O)CC=2C=CON2,0
9,C=CC(=O)NCC=1C=C2OCCCN2N1,0


In [18]:
sol2.SMILES

0          C=CC(=O)NC(=O)NC1(CC2CC3CC(C2)C1)C3
1        CC=1C=CC(=CC1)C(=O)C=2OC=CC2NC(=O)C=C
2               CN(C)C1=NC(Cl)=C(CNC(=O)C=C)S1
3            C=CC(=O)N1CN(CC=2C=CC=CC2)C(=O)C1
4             C=CC(=O)NCC(=O)N1CCC=2C=CC=CC2C1
5     CN(C)S(=O)(=O)C=1C=CC=2CN(CC2C1)C(=O)C=C
6               CN1N=CC=2C=CC(CNC(=O)C=C)=CC12
7           CC(NC(=O)C=C)C(=O)N1CCC=2C=CC=CC12
8        C=CC(=O)N1CCN(CC1)S(=O)(=O)CC=2C=CON2
9                    C=CC(=O)NCC=1C=C2OCCCN2N1
10             C=CC(=O)NCC=1C=CC=C(N1)N2CCOCC2
11         NC(=O)COC=1C=CC=C2N(CCCC12)C(=O)C=C
12               C=CC(=O)NC(C1CCC1)C=2C=CC=CC2
13                     COC=1C=C(CNC(=O)C=C)ON1
14               C=CC(=O)NC(C1CCC1)C=2C=CC=CN2
15           CN1C(N(CC1=O)C(=O)C=C)C=2C=CC=CC2
16          CC(=O)NC=1C=CC=C2CN(CCC12)C(=O)C=C
17               CC1CN(CC=2C=CC=CC2O1)C(=O)C=C
18       CC=1C=CC(=CC1)C(NC(=O)C=C)C=2C=CC=CN2
19                  C=CC(=O)NCC=1C=CC=C2OCOC21
20      CCC1=NN=C(O1)C2CC=3C=CC=CC3CN2C(=O)C=C
21         FC

In [19]:
# Import necessary libraries

from transformers import BertForSequenceClassification, BertTokenizer, AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from sklearn.metrics import precision_score
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


# Load your dataset
sol = pd.read_csv('SMILES-DATA - Equal-1.csv')

# Tokenize the SMILES strings using a BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128  # Define the maximum sequence length

print(tokenizer)

# Preprocess the data for BERT (tokenization, padding, etc.)
tokens = tokenizer(sol['SMILES'].tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

# Extract input_ids and attention_mask from tokens
input_ids = tokens['input_ids']
attention_masks = tokens['attention_mask']

# Convert 'RESULT' column to a tensor
labels = torch.tensor(sol['RESULT'].values)

print('INPUT', input_ids)
print('attention_masks', attention_masks)
print('labels', labels)

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
INPUT tensor([[  101, 10507,  1027,  ...,     0,     0,     0],
        [  101, 27166,  1006,  ...,     0,     0,     0],
        

In [24]:
import joblib
import numpy as np
from sklearn.metrics import precision_score, confusion_matrix, classification_report


# Evaluation loop
model.eval()
predictions, true_labels = [], []
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = batch[2].to('cpu').numpy()
    predictions.extend(np.argmax(logits, axis=1).flatten())
    true_labels.extend(label_ids.flatten())

    batch_predictions = np.argmax(logits, axis=1).flatten()
    
    # New loop to print true labels and predictions for each batch
    for true_label, prediction in zip(label_ids.flatten(), batch_predictions):
        print(f"True Label: {true_label}, Prediction: {prediction}")

# Calculate precision

precision = precision_score(true_labels, predictions, average='binary')
print(f"Precision: {precision}")


True Label: 0, Prediction: 0
True Label: 1, Prediction: 0
True Label: 1, Prediction: 0
True Label: 0, Prediction: 0
True Label: 0, Prediction: 0
True Label: 0, Prediction: 0
True Label: 0, Prediction: 1
True Label: 1, Prediction: 0
True Label: 1, Prediction: 0
True Label: 1, Prediction: 0
True Label: 1, Prediction: 0
True Label: 0, Prediction: 0
True Label: 1, Prediction: 0
True Label: 1, Prediction: 1
True Label: 0, Prediction: 0
True Label: 0, Prediction: 0
True Label: 0, Prediction: 0
True Label: 1, Prediction: 1
True Label: 0, Prediction: 0
True Label: 0, Prediction: 1
True Label: 0, Prediction: 1
True Label: 1, Prediction: 1
True Label: 1, Prediction: 0
True Label: 1, Prediction: 0
True Label: 1, Prediction: 0
True Label: 1, Prediction: 0
True Label: 0, Prediction: 0
True Label: 1, Prediction: 1
True Label: 0, Prediction: 0
True Label: 0, Prediction: 0
True Label: 0, Prediction: 0
True Label: 0, Prediction: 0
True Label: 0, Prediction: 1
True Label: 0, Prediction: 0
True Label: 1,